In [76]:
    import csv
    # read data
    data_name = 'Table_5_1.csv'
    print("Transaction file: ", data_name)
    file = open(data_name, 'r')
    my_reader = csv.reader(file)
    data = []
    for row in my_reader:
        data.append(row)
    
    # data cleansing
    for transaction in data:
        transaction = transaction.pop(0)
    
    # Set hyperparameter (min_sup)
    MIN_SUP = 2
    print("\nmin_sup: ",MIN_SUP)
    
    # print out the number of transactions
    print("\nNumber of transactions: ", len(data))
    
    # create candidate 1-itemset (initialization)
    def create_C1(data):
        C1 = {}
        for transaction in data:
            for item in transaction:
                if item in C1:
                    C1[item] += 1
                else:
                    C1[item] = 1
        return C1 
    
    # function for generating candidates of larger itemsets
    def candidate_generation(Lk, k, data): 
        Lk = list(Lk.keys())
        for i in range(len(Lk)):
            Lk[i] = set(list(Lk[i].split(',')))
        Ck = []
        for i in range(len(Lk)):
            Lk_list = list(Lk[i])
            for p in range(len(Lk_list)):
                testing_subset = set()
                for q in range(len(Lk_list)):
                    if q != p:
                        testing_subset.add(Lk_list[q])
                for j in range(i+1, len(Lk)):# compare each k-itemsets with every other k-itemsets to generate k+1-itemsets with join (disregard infrequent itemsets)
                    if testing_subset.issubset(Lk[j]):
                        Ck.append(Lk[i] | Lk[j]) # join the two itemsets
        Ck_no_duplicate = []
        for itemset in Ck:
            if itemset not in Ck_no_duplicate:
                Ck_no_duplicate.append(itemset)
        return Ck_no_duplicate
    
    # function for pre-pruning candidate itemsets (prune itemsets in Ck with subsets of length k-1 that are infrequent)
    def candidate_pruning(prevLk, Ck):
        Ck_pruned = []
        # convert prevLk to list of lists form
        prevLk_list = list(prevLk.keys())
        for i in range(len(prevLk_list)):
            prevLk_list[i] = list(prevLk_list[i].split(','))
        # consider the subsets of each candidate in Ck
        for itemset in Ck:
            curr_item_list = list(itemset)
            # leave one element out and generate subsets of length k-1
            for i in range(len(curr_item_list)):
                testing_subset = []
                for j in range(len(curr_item_list)):
                    if j != i:
                        testing_subset.append(curr_item_list[j])
                testing_subset.sort()
                # if a subset is not within the k-1-itemset list, prune the full itemset
                flag = 0 # flag for determining whether to prune the candidate itemset
                for item in prevLk_list:
                    item.sort()
                    if testing_subset == item:
                        flag = 1 # subset in the prevLk
                        break
                if flag == 0: # subset not in the prevLk
                    break # current candidate pruned, consider the next candidate
                if i == len(curr_item_list) - 1:
                    Ck_pruned.append(itemset) # if all subsets are within the prevLk, we keep the candidate
        return Ck_pruned
    
    # function for generating the support of each candidate sets
    def generate_sup_count_list(data, Ck_pruned):
        # convert data to list of set form
        data_set = [set() for i in range(len(data))]
        for i in range(len(data)):
            for item in data[i]:
                data_set[i].add(item) 
        # only for first iteration to see the change more clearer
        if type(Ck_pruned) == dict:
            Ck_list = list(Ck_pruned.keys())
            Ck_pruned = [set() for i in range(len(Ck_list))]
            for i in range(len(Ck_pruned)):
                Ck_pruned[i].add(Ck_list[i])
        sup_count = {}
        for transaction in data:
            for candidate in Ck_pruned:
                if candidate.issubset(transaction):
                    candidate = ','.join(str(item) for item in candidate) # convert set to string to search in the dictionary or add a new key in dictionary
                    if candidate in sup_count:
                        sup_count[candidate] += 1
                    else:
                        sup_count[candidate] = 1
#         sup_dic = {}
#         for key in sup_count:
#             sup = float(sup_count[key]) / float(len(data))
#             sup_dic[key] = sup
        return sup_count
    
    # function for generating frequent itemsets (transformation from Ck to Lk)
    def candidate_elimination(min_sup, sup_count):
        Lk = {}
        for key in sup_count:
            if sup_count[key] >= min_sup:
                Lk[key] = sup_count[key]
        return Lk
    
    # main function for a priori algorithm
    def apriori(data, min_sup):
        C1 = create_C1(data) # initialization
        print("\nCandidate %d-itemset has %d items." % (1, len(C1)))
        print(C1)
        print("\nPruned %d-itemset is the same as the eliminated %d-itemset." % (1, 1))
        sup_dic = generate_sup_count_list(data, C1) # only support generation and candidate elimination is required for C1
        L1 = candidate_elimination(MIN_SUP, sup_dic)
        print("\nEliminated %d-itemset has %d items" % (1,len(L1)))
        print(L1)
        L = [L1] # create a list of filtered itemsets so we can track the previous sets for arguments such as prevLk
        k = 2 # now, we want to generate 2-itemset

        # run loop to continuously generate Ck and filter to Lk until empty set is reached
        while len(L[k-2]) > 0:
            Ck = candidate_generation(L[k-2], k, data) # candidate generation
            print("\nCandidate %d-itemset has %d items." % (k, len(Ck)))
            print(Ck)
            Ck_pruned = candidate_pruning(L[k-2], Ck) # candidate pruning
            print("\nPruned %d-itemset has %d items." % (k,len(Ck_pruned)))
            print(Ck_pruned)
            sup_dic = generate_sup_count_list(data, Ck_pruned) # support count dictionary generation
            Lk = candidate_elimination(MIN_SUP, sup_dic) # candidate elimination
            print("\nEliminated %d-itemset has %d items." % (k,len(Lk)))
            print(Lk)
            L.append(Lk)
            k += 1
        print("END")
        return 
    
    # Driver Code
    apriori(data, MIN_SUP)
                    
        
    
                

Transaction file:  Table_5_1.csv

min_sup:  2

Number of transactions:  5

Candidate 1-itemset has 6 items.
{'Bread': 4, 'Milk': 4, 'Diapers': 4, 'Beer': 3, 'Eggs': 1, 'Cola': 2}

Pruned 1-itemset is the same as the eliminated 1-itemset.

Eliminated 1-itemset has 5 items
{'Bread': 4, 'Milk': 4, 'Diapers': 4, 'Beer': 3, 'Cola': 2}

Candidate 2-itemset has 10 items.
[{'Bread', 'Milk'}, {'Diapers', 'Bread'}, {'Bread', 'Beer'}, {'Cola', 'Bread'}, {'Diapers', 'Milk'}, {'Beer', 'Milk'}, {'Cola', 'Milk'}, {'Diapers', 'Beer'}, {'Diapers', 'Cola'}, {'Cola', 'Beer'}]

Pruned 2-itemset has 10 items.
[{'Bread', 'Milk'}, {'Diapers', 'Bread'}, {'Bread', 'Beer'}, {'Cola', 'Bread'}, {'Diapers', 'Milk'}, {'Beer', 'Milk'}, {'Cola', 'Milk'}, {'Diapers', 'Beer'}, {'Diapers', 'Cola'}, {'Cola', 'Beer'}]

Eliminated 2-itemset has 8 items.
{'Bread,Milk': 3, 'Diapers,Bread': 3, 'Bread,Beer': 2, 'Diapers,Beer': 3, 'Diapers,Milk': 3, 'Beer,Milk': 2, 'Cola,Milk': 2, 'Diapers,Cola': 2}

Candidate 3-itemset has 9 i